We use polars to efficiently analyze the csv 

In [10]:
import polars as pl

The parsed message will get the colukmn names as defined in the clean.csv

In [11]:
MESSAGE_HEADERS = ["status", "lat", "lat_dir", "lon", "lon_dir", "spd_over_grnd", "true_course", "datestamp",
                   "mag_variation", "mag_var_dir"]

We next read the raw_messages.csv, split the raw message column and filter out all records that have a number of message parts not equal to 
```python
len(MESSAGE_HEADERS)
```

In [12]:
df = pl.read_csv('raw_messages.csv', has_header=True)
print(f' the csv file has shape {df.shape}')

columns = df.columns[:-1]
message = df.columns[-1]

df = (df
      .with_columns(pl.col(message).str.split(',').alias(message))
      .filter(pl.col(message).list.lengths() == len(MESSAGE_HEADERS))  # nothing was filtered out
      .with_columns(pl.col(message).list.to_struct())
      .unnest(message)
      .drop(message))
df.columns = columns + MESSAGE_HEADERS
print(f'after separating the message details the shape is {df.shape}')

 the csv file has shape (29052, 6)
after separating the message details the shape is (29052, 15)


looks good, apparently the number of parts of the message is correct for all entries. Now check if the numeric values are indeed numeric and the categorical entries have their predefined values

In [4]:
corrupt_df = (df
      .filter(pl.col("status").is_in(["V", "A"]))
      .filter(pl.col("lat_dir").is_in(["N", "S"]))
      .filter(pl.col("lon_dir").is_in(["E", "W"]))
      .filter(pl.col("mag_var_dir").is_in(["E", "W"]))
      .filter(pl.col("lat").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("lon").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("spd_over_grnd").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("true_course").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("datestamp").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("mag_variation").str.contains(r'^\d*\.?\d+'))
      )
print(corrupt_df.shape)


(20072, 15)


there are some records with errors in their message. We can filter all non desired characters out

In [13]:
final_df = (df
      .with_columns(pl.col("status").str.replace_all(r"[^VA]", ""))
      .with_columns(pl.col("lon_dir").str.replace_all(r"[^EW]", ""))
      .with_columns(pl.col("lat_dir").str.replace_all(r"[^NS]", ""))
      .with_columns(pl.col("mag_var_dir").str.replace_all(r"[^EW]", ""))
      .with_columns(pl.col("lat").str.replace_all(r"[^0-9.]", ""))
      .with_columns(pl.col("lon").str.replace_all(r"[^0-9.]", ""))
      .with_columns(pl.col("lat").str.replace_all(r"[^0-9.]", ""))
      .with_columns(pl.col("spd_over_grnd").str.replace_all(r"[^0-9.]", ""))
      .with_columns(pl.col("true_course").str.replace_all(r"[^0-9.]", ""))
      .with_columns(pl.col("mag_variation").str.replace_all(r"[^0-9.]", ""))
      .filter(pl.col("status").is_in(["V", "A"]))
      .filter(pl.col("lat_dir").is_in(["N", "S"]))
      .filter(pl.col("lon_dir").is_in(["E", "W"]))
      .filter(pl.col("mag_var_dir").is_in(["E", "W"]))
      .filter(pl.col("lat").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("lon").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("spd_over_grnd").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("true_course").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("datestamp").str.contains(r'^\d*\.?\d+'))
      .filter(pl.col("mag_variation").str.contains(r'^\d*\.?\d+'))
      )
print(final_df.shape)

(29052, 15)


that looks ok, there was just some noise in the messages.


In [14]:
print(final_df)

shape: (29_052, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ device_id ┆ datetime  ┆ address_i ┆ address_p ┆ … ┆ true_cour ┆ datestamp ┆ mag_varia ┆ mag_var_ │
│ ---       ┆ ---       ┆ p         ┆ ort       ┆   ┆ se        ┆ ---       ┆ tion      ┆ dir      │
│ str       ┆ i64       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ str       ┆ ---       ┆ ---      │
│           ┆           ┆ str       ┆ i64       ┆   ┆ str       ┆           ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 0001      ┆ 155006699 ┆ 172.19.0. ┆ 4007      ┆ … ┆ 1.59      ┆ 150218    ┆ 0.8       ┆ E        │
│           ┆ 9         ┆ 17        ┆           ┆   ┆           ┆           ┆           ┆          │
│ 0001      ┆ 155006766 ┆ 172.19.0. ┆ 4007      ┆ … ┆ 1.59      ┆ 150218    ┆ 0.8       ┆ E        │
│           ┆ 1         ┆ 16        ┆           ┆   ┆           ┆      

The datetime is in unix time we want it in human readable format.

No need to convert the message columns to their actual data types, since we are exporting to csv  anyhow

In [16]:
final_df.write_csv('cleaned_messages.csv', has_header=True, datetime_format='%F %T')